# API Gateway を AgentCore Gateway ターゲットとして統合

## 概要

組織がエージェント型アプリケーションの可能性を探求する中で、大規模言語モデル（LLM）への呼び出しリクエストのコンテキストとしてエンタープライズデータを安全に、エンタープライズポリシーに沿った方法で使用するという課題に直面し続けています。これらのインタラクションを標準化し安全にするために、多くの組織はエージェント型アプリケーションがデータソースやツールに安全に接続する方法を定義する Model Context Protocol（MCP）仕様を使用しています。

MCP は新しいユースケースに有利ですが、組織は既存の API エステートをエージェント時代に導入するという課題にも直面しています。MCP は確かに既存の API をラップできますが、追加作業が必要です。MCP から RESTful API へのリクエストの変換、リクエストフロー全体を通じてセキュリティが維持されていることの確認、本番デプロイメントに必要な標準的な可観測性の適用などです。

[Amazon Bedrock AgentCore Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) が [Amazon API Gateway](https://aws.amazon.com/api-gateway/) をターゲットとしてサポートするようになり、AgentCore Gateway（ACGW）への MCP リクエストを API Gateway（APIGW）への RESTful リクエストに変換します。APIGW からの新規および既存の API エンドポイントを、組み込みのセキュリティと可観測性を備えた MCP を通じてエージェント型アプリケーションに公開できるようになりました。このノートブックではこの新機能をカバーし、実装方法を示します。

## 新機能

AgentCore Gateway は、Lambda 関数、OpenAPI スキーマ、Smithy モデル、MCP サーバーなど、すでに複数のターゲットタイプをサポートしており、現在は API Gateway もサポートしています。


![](Images/agent-core-gateway-targets.png)


**お客様は API Gateway を使用して、多数のアプリケーションにまたがるバックエンドを接続する広範な API エコシステムを正常に構築しています。** 企業が次世代のエージェント型アプリケーションに進むにつれ、自然な進化として、これらの既存の API とバックエンドツールを AI 駆動システムに公開し、確立されたインフラストラクチャと最新のインテリジェントエージェント間のシームレスな統合を可能にすることです。

現在、お客様は APIGW API を OpenAPI 3 仕様としてエクスポートし、それを OpenAPI ターゲットとして ACGW に追加する手動ワークフローに従っています。この統合は、APIGW と ACGW 間の接続を自動化することでこのプロセスを合理化することを目的としています。


この統合により、お客様はこのエクスポート/インポートプロセスを自分で管理する必要がなくなります。新しい API_GATEWAY ターゲットタイプが ACGW に追加されます。REST API 所有者は、数回のコンソールクリックまたは単一の CLI コマンドで API を ACGW ターゲットとして追加でき、既存の REST API メソッドを ACGW 経由で MCP ツールとして公開できます。API 消費者は、Model Context Protocol（MCP）を通じて AI エージェントをこれらの REST API に接続し、AI 統合でワークフローを強化できます。エージェント型アプリケーションが新規または既存の APIGW API に接続できるようになりました。現在、ACGW と APIGW 間のこの統合は、IAM 認可と API キー認可をサポートしています。

![](Images/agent-core-apigw-target.png)

### チュートリアルの詳細


| 情報                | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                          |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity                  |
| Agentic フレームワーク | Strands Agents                                          |
| Gateway ターゲットタイプ | API Gateway                                            |
| エージェント         | Strands                                                   |
| インバウンド認証 IdP | Amazon Cognito、ただし他も使用可能                        |
| アウトバウンド認証   | IAM 認可と API キー                                       |
| LLM モデル           | Anthropic Claude Sonnet 4                                 |
| チュートリアルコンポーネント | AgentCore Gateway ターゲット経由での API Gateway 呼び出し |
| チュートリアル分野   | クロスバーティカル                                        |
| 難易度               | 簡単                                                      |
| 使用 SDK             | boto3                                                     |

## チュートリアルアーキテクチャ

このチュートリアルは、より広範なエンタープライズ課題の実践的な例として機能します：**次世代エージェント型アプリケーションのために API Gateway API を一元化された Gateway アーキテクチャに統合する方法。**

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 資格情報
* Amazon Cognito

In [ ]:
# Install from the requirements file or pyproject.toml file in current directory
!pip3 install --force-reinstall -U -r requirements.txt --quiet

# Feel free to modify it depending upon the environment you are using. The following command might help.
# !uv pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-west-2')

In [ ]:
# Import utils
import os
import sys
import importlib

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# utils.py is in the same directory as the notebook
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Import utils and reload to pick up any changes
import utils
importlib.reload(utils)

# Setup logging 
import logging

# Configure logging for notebook environment
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# Set specific logger levels
logging.getLogger("strands").setLevel(logging.INFO)

## インバウンドおよびアウトバウンド認可の前提条件のセットアップ

- インバウンド認可は、受信ユーザーリクエストを認証します。
- アウトバウンド認可は、ACGW が認証されたユーザーに代わって、APIGW などの Gateway ターゲットに安全に接続するのを支援します。

![](Images/agent-core-auth.png)
 
API Gateway をターゲットとする場合、ACGW は以下のタイプのアウトバウンド認可をサポートします：
-	認可なし（非推奨）– 一部のターゲットタイプでは、アウトバウンド認可をバイパスするオプションを提供しています。この安全性の低いオプションはお勧めしません。

-	IAM ベースのアウトバウンド認可 – AWS Signature Version 4（Sig V4）で Gateway ターゲットへのアクセスを認可するために [Gateway サービスロール](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-prerequisites-permissions.html#gateway-service-role-permissions)を使用します。

-	API キー – API Gateway で構成され、AgentCore Identity でセットアップされた API キーを使用します。APIGW 経由で作成された API キーは APIGW 使用量プランにマッピングされ、監視と制御に役立ちます。詳細については、この[ドキュメント](https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-api-usage-plans.html)を参照してください。

IAM ベースの認可によるアウトバウンド認可の場合、ポリシーには "execute-api:Invoke" 権限を含める必要があります。

#### ACGW が引き受ける IAM ロールをすばやく作成しましょう。
**この IAM ロールは、APIGW を呼び出す際の IAM ベースの認可にも使用します。**

In [ ]:
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-agent-core-gateway-role-for-api-gateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

## Amazon Cognito を使用した Gateway へのインバウンド認証の構成

AgentCore Gateway への受信リクエストの JWT ベースの認証を処理する Cognito ユーザープールを作成します。

**作成されるコンポーネント：**
- ID 管理用のユーザープール
- OAuth 2.0 スコープ用のリソースサーバー
- プログラムによるアクセス用のマシン間（M2M）クライアント

In [ ]:
# Creating Cognito User Pool 
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "invoke",  # Just 'invoke', will be formatted as resource_server_id/invoke
    "ScopeDescription": "Scope for invoking the agentcore gateway"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {gw_user_pool_id}")

utils.get_or_create_resource_server(cognito, gw_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(cognito, gw_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"Client ID: {gw_client_id}")

# Get discovery URL
gw_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration'
print(gw_cognito_discovery_url)

## Gateway の作成
Agent Core Gateway を作成しましょう

In [ ]:
# CreateGateway with Cognito authorizer. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [gw_client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": gw_cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='sample-ac-gateway-apigw-demo',
    roleArn=agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway
    protocolType='MCP',
    protocolConfiguration={
        'mcp': {
            'supportedVersions': ['2025-03-26'],
            'searchType': 'SEMANTIC'
        }
    },
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway with API Gateway target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

## サンプル PetStore API を Amazon API Gateway にデプロイ
### API 概要

IAM と API キー認可パターンの両方を示すサンプル PetStore REST API をデプロイします。

**API エンドポイント：**

| エンドポイント | メソッド | 認可 | 説明 |
|:---------|:-------|:--------------|:------------|
| `/pets` | GET | IAM（SigV4） | 利用可能なすべてのペットを一覧表示 |
| `/pets` | POST | IAM（SigV4） | 新しいペットを追加 |
| `/pets/{petId}` | GET | IAM（SigV4） | ID でペットを取得 |
| `/orders/{orderId}` | GET | API キー | 注文の詳細を取得 |

**作成されるリソース：**
- モック統合を持つ REST API
- `/orders` エンドポイント用の API キー
- レート制限付きの使用量プラン
- `dev` ステージへのデプロイ

OpenAPI 仕様は [AgentCore_Sample_API-dev-oas30-apigateway.json](AgentCore_Sample_API-dev-oas30-apigateway.json) で利用可能です。

In [ ]:
# Load OpenAPI definition with API Gateway extensions, import to API Gateway, and deploy
# This uses the OpenAPI document that already has integrations and security configured
api_result = utils.create_and_deploy_api_from_openapi_with_extensions(
    filename='AgentCore_Sample_API-dev-oas30-apigateway.json',
    stage_name='dev',
    description='Initial Deployment'
)

# Extract the API details for use in subsequent cells
api_id = api_result['api_id']
api_key = api_result['api_key_value']
invoke_url=api_result['invoke_url']

print(f"\n{'='*70}")
print(f"API Details for Gateway Target Configuration")
print(f"{'='*70}")
print(f"API ID: {api_id}")
print(f"API Name: {api_result['api_name']}")
print(f"Stage: {api_result['stage_name']}")
print(f"API Key ID: {api_result['api_key_id']}")
print(f"Usage Plan ID: {api_result['usage_plan_id']}")
print(f"{'='*70}")

## API Gateway デプロイの検証

適切な構成と認可を検証するために、すべてのエンドポイントをテストします。

**注意：** テストを実行する前に、API Gateway の変更がアベイラビリティゾーン全体に伝播するまで 10〜15 秒待ってください。

In [ ]:
test_results = utils.test_api_gateway_endpoints(
    invoke_url=api_result['invoke_url'],
    api_key=api_result['api_key_value'],
    region=os.environ.get('AWS_DEFAULT_REGION', 'us-west-2')
)

# Display detailed results
print("\nDetailed Test Results:")
for endpoint, result in test_results.items():
    print(f"\n{endpoint}: {result['status']}")
    if 'data' in result:
        print(f"  Data: {result['data']}")
    if 'error' in result:
        print(f"  Error: {result['error']}")



## AgentCore Identity で生成された API キーを保護
以下のセクションでは、AgentCore Identity に新しい API キー資格情報プロバイダーを作成します。これは、後で `/orders` エンドポイント用の AgentCore Gateway ターゲットを作成する際に使用されます。

API キーは AgentCore Identity 経由で AWS Secrets Manager に安全に保存されます。

In [ ]:
import boto3
import os
credentialProviderArn=""
try:
    # Create Bedrock Agent Core Control client
    REGION = os.environ['AWS_DEFAULT_REGION']
    bedrock_agent_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
    
    # Create API key credential provider in AgentCore identity
    credential_provider_response = bedrock_agent_client.create_api_key_credential_provider(
        name='sample-api-gateway-key-provider',
        apiKey=api_key,
        tags= {
            'ProjectName':'AgentCore-APIGW-Sample-Application'
        }
    )

    # Store the ARNs in variables
    secretArn = credential_provider_response['apiKeySecretArn']['secretArn']
    credentialProviderArn = credential_provider_response['credentialProviderArn']
    print(f"Credential Provider Arn: {credentialProviderArn}")
    
except KeyError as e:
    print(f"Environment variable or response key missing: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


## Gateway ターゲットの作成
API Gateway ターゲットを作成するには、ターゲット構成の一部として以下を指定する必要があります：
- **toolFilters**: REST API のどのリソースを ACGW のツールとして公開するかを決定するために使用します。フィルターは filterPath でワイルドカードもサポートしています。
- **toolOverrides**（オプション）: ユーザーがツール名と説明をオーバーライドできるようにするために使用します。明示的なパスとメソッドを指定する必要があります。指定されていない場合、APIGW からの説明を使用します。
- **restApiId**: API Gateway ID を渡すために使用します。
- **stage**: API Gateway のデプロイ済みステージ名を渡すために使用します

![](Images/agent-core-apigw-target.png)

以下はターゲット構成の例です：

### 例 1：

これは "GET & POST /pets"、"GET /pets/{petId}" を Gateway に公開し、ツール名と説明をオーバーライドします。

In [ ]:
{
  "mcp": {
    "apiGateway": {
      "restApiId": "<api-id>",
      "stage": "<stage>",
      "apiGatewayToolConfiguration": {
        "toolFilters": [
          {
            "filterPath": "/pets",
            "methods": ["GET","POST"]
          },
          {
            "filterPath": "/pets/{petId}",
            "methods": ["GET"]
          }
        ],
      "toolOverrides" : [
          {
             "name": "ListPets",
             "path": "/pets",
             "method": "GET",
             "description":"Retrieves all the available Pets."
         },
         {
              "name": "AddPet",
              "path": "/pets",
              "method": "POST",
               "description":"Add a new pet to the available Pets."
          },
          {
             "path": "/pets/{petId}",
             "method": "GET",
             "name": "GetPetById",
             "description": "Retrieve a specific pet by its ID"
         }
         ]	
      }
    }
  }
}

### 例 2
これは "GET /pets" だけでなく "GET /pets/{petId}" または "/pets" 配下のすべてを公開します。toolOverrides が指定されていないため、API Gateway からのリソース説明を使用します。

In [ ]:
{
  "mcp": {
    "apiGateway": {
      "restApiId": "<api-id>",
      "stage": "<stage>",
      "apiGatewayToolConfiguration": {
        "toolFilters": [
          {
            "filterPath": "/pets/*",
            "methods": ["GET"]
          }
        ]
      }
    }
  }
}


## Gateway ターゲット 1 の作成：IAM 認可エンドポイント

このターゲットは、IAM 認可を使用して `/pets` と `/pets/{petId}` エンドポイントを公開します。

In [ ]:
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

# creating Gateway Target for '/pets' and '/pets/{petId}' resource with IAM credential provider
create_gateway_target_1_response = gateway_client.create_gateway_target(
    name='api-gateway-target-1',
    gatewayIdentifier=gatewayID,
    targetConfiguration=
            {
            "mcp": {
                "apiGateway": {
                "restApiId": api_id,
                "stage": "dev",
                "apiGatewayToolConfiguration": {
                    "toolFilters": [
                    {
                        "filterPath": "/pets",
                        "methods": ["GET","POST"]
                    },
                    {
                        "filterPath": "/pets/{petId}",
                        "methods": ["GET"]
                    }
                    ],
                    "toolOverrides" : [
                    {
                        "name": "List_Pets",
                        "path": "/pets",
                        "method": "GET",
                        "description":"Retrieves all the available Pets."
                    },
                    {
                        "name": "Add_Pet",
                        "path": "/pets",
                        "method": "POST",
                        "description":"Add a new pet to the available Pets."
                    },
                    {
                        "path": "/pets/{petId}",
                        "method": "GET",
                        "name": "GetPetById",
                        "description": "Retrieve a specific pet by its ID"
                    }
                    ]
                    }
                }
                }
            },
    credentialProviderConfigurations=[
        {
        "credentialProviderType": "GATEWAY_IAM_ROLE"
        }
    ]
)
print(f"Create Gateway Target 1 Response: {create_gateway_target_1_response}")
gateway_target1_id=create_gateway_target_1_response['targetId']

## Gateway ターゲット 2 の作成：API キー認可エンドポイント

このターゲットは、以前に作成した資格情報プロバイダーからの API キー認可を使用して `/orders/{orderId}` エンドポイントを公開します。

In [ ]:
# Creating 2nd Gateway Target for /orders/{orderId} resource with API Key credential provider
create_gateway_target_2_response = gateway_client.create_gateway_target(
    name='api-gateway-target-2',
    gatewayIdentifier=gatewayID,
    targetConfiguration=
            {
            "mcp": {
                "apiGateway": {
                "restApiId": api_id,
                "stage": "dev",
                "apiGatewayToolConfiguration": {
                    "toolFilters": [
                    {
                        "filterPath": "/orders/{orderId}",
                        "methods": ["GET"]
                    }
                    ],
                    "toolOverrides" : [
                    {
                        "path": "/orders/{orderId}",
                        "method": "GET",
                        "name": "GetOrderById",
                        "description": "Retrieve a specific order by its ID"
                    }
                    ]
                    }
                }
                }
            },
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "API_KEY",
            "credentialProvider": {
                "apiKeyCredentialProvider": {
                    "providerArn": credentialProviderArn,
                    "credentialParameterName": "x-api-key", 
                    "credentialLocation": "HEADER" 
                }
            }
        }
    ]
)
print(f"Create Gateway Target 2 Response: {create_gateway_target_2_response}")
gateway_target2_id=create_gateway_target_2_response['targetId']

## Gateway ターゲットステータスの確認

続行する前に、両方のターゲットが `READY` 状態であることを確認してください。いずれかのターゲットが `FAILED` ステータスを示す場合は、[get_gateway_target API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control/client/get_gateway_target.html) を使用して失敗の理由を調査してください。

In [ ]:
list_gateway_targets_response = gateway_client.list_gateway_targets(
    gatewayIdentifier=gatewayID,
    maxResults=10
)
for i, item in enumerate(list_gateway_targets_response['items'], 1):
    print(f"Target {i}:")
    print(f"  Target ID: {item['targetId']}")
    print(f"  Name: {item['name']}")
    print(f"  Status: {item['status']}")
    print()


## AI エージェントとの統合テスト

Strands AI フレームワークを使用して、以下により完全な統合をテストします：
1. Gateway から利用可能なツールを一覧表示
2. ペット関連エンドポイントを呼び出し（IAM 認証）
3. 注文エンドポイントを呼び出し（API キー認証）

In [ ]:
import json

import requests
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent


def get_token():
    token = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
    return token['access_token']


def create_streamable_http_transport():
    return streamablehttp_client(
        gatewayURL, headers={"Authorization": f"Bearer {get_token()}"}
    )


client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", # may need to update model_id depending on region
    temperature=0.7,
    max_tokens=500,  # Limit response length
)

with client:
    # Call the listTools
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools

    agent = Agent(model=yourmodel, tools=tools)  
    ## you can replace with any model you like
    
    print("\n" + "="*70)
    print("Test 1: List Available Tools")
    print("="*70)
    agent("List all tools available to you")
    
    print("\n" + "="*70)
    print("Test 2: List All Pets (IAM Auth)")
    print("="*70)
    agent("List all the available pets")
    
    print("\n" + "="*70)
    print("Test 3: Get Pet by ID (IAM Auth)")
    print("="*70)
    agent("Tell me about the pet with petId 3")
    
    print("\n" + "="*70)
    print("Test 4: Get Order Details (API Key Auth)")
    print("="*70)
    agent("When will my order be delivered? My order id is 2")



## リソースのクリーンアップ

以下のセルを実行して、このチュートリアルで作成されたすべてのリソースを削除します：

**削除されるリソース：**
1. AgentCore Gateway とターゲット
2. AgentCore Identity 資格情報プロバイダー
3. API Gateway REST API、API キー、使用量プラン
4. Amazon Cognito ユーザープールとクライアント
5. IAM ロール

In [ ]:
# Delete AgentCore Gateway and all targets
print("Step 1: Cleaning up AgentCore Gateway resources...")
agentcore_cleanup = utils.delete_agentcore_gateway_and_targets(
    gateway_id=gatewayID,
    region=REGION
)

# Delete AgentCore Identity Credential Provider
print("\nStep 2: Cleaning up AgentCore Identity Credential Provider...")
credential_cleanup = utils.delete_agentcore_credential_provider(
    credential_provider_arn=credentialProviderArn,
    region=REGION
)

# Delete API Gateway and all related resources
print("\nStep 3: Cleaning up API Gateway resources...")
api_cleanup = utils.delete_api_gateway_and_resources(
    api_id=api_id,
    api_key_id=api_result.get('api_key_id'),
    usage_plan_id=api_result.get('usage_plan_id')
)

# Delete Cognito User Pool
print("\nStep 4: Cleaning up Cognito User Pool...")
cognito_cleanup = utils.delete_cognito_user_pool(
    user_pool_name='sample-agentcore-gateway-pool',
    region=REGION
)

# Delete IAM Role
print("\nStep 5: Cleaning up IAM Role...")
iam_cleanup = utils.delete_iam_role(
    role_name='agentcore-sample-agent-core-gateway-role-role'
)

# Overall Summary
print("\n" + "="*70)
print("Overall Cleanup Summary")
print("="*70)
print(f"AgentCore Gateway Deleted: {'✓' if agentcore_cleanup['gateway_deleted'] else '✗'}")
print(f"AgentCore Targets Deleted: {len(agentcore_cleanup['targets_deleted'])}")
print(f"Credential Provider Deleted: {'✓' if credential_cleanup['credential_provider_deleted'] else '✗'}")
print(f"Cognito User Pool Deleted: {'✓' if cognito_cleanup['user_pool_deleted'] else '✗'}")
print(f"Cognito Clients Deleted: {len(cognito_cleanup['clients_deleted'])}")
print(f"IAM Role Deleted: {'✓' if iam_cleanup['role_deleted'] else '✗'}")
print(f"IAM Policies Deleted: {len(iam_cleanup['policies_deleted'])}")
print(f"API Gateway Deleted: {'✓' if api_cleanup['api_deleted'] else '✗'}")
print(f"API Key Deleted: {'✓' if api_cleanup.get('api_key_deleted') else '✗'}")
print(f"Usage Plan Deleted: {'✓' if api_cleanup.get('usage_plan_deleted') else '✗'}")
print("="*70)

## サマリー
AgentCore Gateway と API Gateway 間の新しい統合を正常にテストしました。これにより、既存の REST API を次世代エージェント型アプリケーション用の MCP 互換エンドポイントとして公開できるようになりました。